In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!mkdir /content/drive/MyDrive/Periodic_Motion_CV/frames

In [9]:
import cv2

# Path to the input video file
video_path = '/content/drive/MyDrive/Periodic_Motion_CV/walking.mp4'

# Path to the output frames directory
output_directory = '/content/drive/MyDrive/Periodic_Motion_CV/frames'

# Open the video file
video = cv2.VideoCapture(video_path)

# Initialize variables
frame_count = 0
success = True

# Iterate through the video frames
while success:
    # Read the next frame
    success, frame = video.read()
    
    if success:
        # Construct the output frame file name
        frame_filename = f'frame_{frame_count}.jpg'
        
        # Save the frame as an image file
        frame_path = os.path.join(output_directory, frame_filename)
        cv2.imwrite(frame_path, frame)
        
        # Increment the frame count
        frame_count += 1

# Release the video file
video.release()

print(f'Frames extracted: {frame_count}')

Frames extracted: 356


<h2>Detect image points of toes on ground plane </h2> 

<h3>Detect toe positions on pedestrian blobs</h3> 

In [ ]:
import cv2
import numpy as np

# Function to extract pixels, center and the eigenvector from the bounding box
def extract_eigenvector(image, bounding_box):
    #'bounding_box' is the YOLO-detected bounding box
    x, y, w, h = bounding_box
    
    # Get the shape of the image
    height, width, _ = image.shape
    # Convert normalized coordinates to pixel coordinates
    x = int(x * width)
    y = int(y * height)

    c=(x,y)

    roi = image[y-h/2:y+h/2, x-w/2:x+w/2]
    pixels = np.argwhere(roi > 0) + np.array([x, y])

    # Compute the mean of the pixel coordinates
    mean = np.mean(pixels, axis=0)

    # Compute the centered coordinates by subtracting the mean
    centered_pixels = pixels - mean

    # Compute the covariance matrix
    covariance_matrix = np.cov(centered_pixels, rowvar=False)

    # Compute the eigenvalues and eigenvectors of the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

    # Sort the eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Access the first eigenvector (e) and its corresponding eigenvalue (v1)
    e = eigenvectors[:, 0]

    return pixels, c, e

In [ ]:
#Obtain the lambda-shaped frames
#Obtain the pedestrian blobs from the lambda-shaped frames


def detect_toe_positions(image, bounding_box):
    toe_positions = []
    
    pixels, c, e = extract_eigenvector(image, bounding_box)
        
    # Iterate through each pixel in the blob
    min_dot_product = np.inf
    optimal_toe_position = None
        
    for pixel in pixels:
        # Calculate the vector from c to the pixel
        t = pixel - c
        
        # Calculate the dot product of e and t
        dot_product = np.dot(e, t)
            
        # Check if the dot product is minimal
        if dot_product < min_dot_product:
            min_dot_product = dot_product
            optimal_toe_position = pixel
        elif dot_product == min_dot_product:
            # Choose the pixel farthest from the principal axis
            if np.linalg.norm(pixel - c) > np.linalg.norm(optimal_toe_position - c):
                optimal_toe_position = pixel
        
    # Append the optimal toe position to the list
    toe_positions.append(optimal_toe_position)
    
    return toe_positions


# Example usage:
# Assuming you have pedestrian blobs detected and stored in a list called 'pedestrian_blobs'

# Call the function to detect toe positions
toe_positions = detect_toe_positions(image, bounding_box)

# Print the detected toe positions
for i, position in enumerate(toe_positions):
    print(f"Toe position for pedestrian {i+1}: {position}")
